In [382]:
import numpy as np
import pandas as pd
import requests
import config
import os
import json
import math
from pandas.io.json import json_normalize
from copy import deepcopy

##Schedule_a API guide: https://api.open.fec.gov/developers/#/receipts/get_schedules_schedule_a_

In [383]:
### MULTI-CANDIDATE SEARCH ###
# DEVELOPING LOOP FOR CANDIDATE IDS #

#Set search for all 2020 (two-year transaction period) Maine candidates
cand_state = 'ME'
period='2020'
parameters = {'election_year':period
            ,'state':cand_state
            ,'api_key':config.api_key}
#To iterate over comm_ids
id=0
#Change ID to an integer
id=int(id)

#Requests candidate info
r_cands = requests.get('https://api.open.fec.gov/v1/candidates/search',params=parameters).json()
r_cands

#Locates and sets Committee ID from 'principal_committees' sub-array
comm_ids=json_normalize(data=r_cands['results'],record_path='principal_committees')[['committee_id','name','party']]
comm_ids=comm_ids['committee_id'].tolist()
comm_ids

['C00710087',
 'C00314575',
 'C00709899',
 'C00653816',
 'C00703413',
 'C00706739',
 'C00683193',
 'C00433391',
 'C00709279',
 'C00694364']

In [384]:
json_normalize(data=r_cands['results'],record_path='principal_committees')[['committee_id','name','party']]

,committee_id,name,party
0,C00710087,MICHAEL BUNKER FOR MAINE,DEM
1,C00314575,COLLINS FOR SENATOR,REP
2,C00709899,SARA GIDEON FOR MAINE,DEM
3,C00653816,COMMITTEE TO ELECT JARED GOLDEN,DEM
4,C00703413,FRIENDS OF BRE KIDMAN,DEM
5,C00706739,DEREK FOR U.S. SENATE,REP
6,C00683193,DR CAT LONDON FOR US SENATE,DEM
7,C00433391,PINGREE FOR CONGRESS,DEM
8,C00709279,SWEET FOR SENATE,DEM
9,C00694364,DANIELLE VANHELSING FOR SENATE,IND


In [385]:
#Initialize dataframe collector
dfs=[]
id=int(0)

#Loop through pages
for x in range(len(comm_ids[0])):
    
    print('FOR')
    #Initialize query dict
    querydict = {'per_page':'100'
                  ,'committee_id':comm_ids[id]
                  ,'sort':'contribution_receipt_date'
                  ,'api_key':config.api_key
                  ,'is_individual':'true'
                  ,'two_year_transaction_period':period
                 }

    r = requests.get('https://api.open.fec.gov/v1/schedules/schedule_a/',params=querydict).json()
    
    #Results rows
    df = json_normalize(r['results'])
    dfs.append(df)
    
    #Last page variables
    if r['pagination']['last_indexes'] is None:
        None
        if id<len(comm_ids[0]):
                id=id+1
        else:
            break
    else: 
        last_index=pd.to_numeric(r['pagination']['last_indexes']['last_index'])
        last_date=r['pagination']['last_indexes']['last_contribution_receipt_date']
        
        #Loop through pages
        while id<len(comm_ids[0]):

            print('WHILE')

            #Initialize query, without last_index and date
            querydict.update([('last_index',last_index)
                            ,('last_contribution_receipt_date',last_date)])

            r = requests.get('https://api.open.fec.gov/v1/schedules/schedule_a/',params=querydict).json()

            #Results rows
            df = json_normalize(r['results'])
            dfs.append(df)

            #Update Pagination variables
            last_index=pd.to_numeric(r['pagination']['last_indexes']['last_index'])
            last_date=r['pagination']['last_indexes']['last_contribution_receipt_date']        
            dfs.append(df)
            
            #Increment to max ID in list
            if id<len(comm_ids[0]):
                id=id+1
            else:
                break

#Concatenate all dfs
contribs=pd.concat(dfs,sort=True)
contribs=contribs.drop_duplicates(subset='transaction_id')

FOR
FOR
WHILE
WHILE
WHILE
WHILE
WHILE
WHILE
WHILE
WHILE
FOR


In [386]:
#Write itemized individual results to a CSV
cwd = os.getcwd()
contribs.to_csv(cwd+'/maine-fed-itemized-receipts.csv')